In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
pd.set_option('display.max_rows', None)

# Load the customer dataset from the CSV file in the 'data' folder
df = pd.read_csv('COVID-19_Case_Surveillance_Public_Use_Data_groupby_death.csv')

In [2]:
df.head()

,cdc_case_earliest_dt,cdc_report_dt,death_yn
0,1/1/2020,96,Missing
1,1/1/2020,137,No
2,1/1/2020,42,Unknown
3,1/1/2020,468,Yes
4,1/2/2020,91,Missing


In [3]:
df = df[(df.death_yn == 'Yes') | (df.death_yn == 'No')]

In [4]:
df.head()

,cdc_case_earliest_dt,cdc_report_dt,death_yn
1,1/1/2020,137,No
3,1/1/2020,468,Yes
5,1/2/2020,59,No
7,1/2/2020,9,Yes
9,1/3/2020,61,No


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2775 entries, 1 to 5550
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   cdc_case_earliest_dt  2775 non-null   object
 1   cdc_report_dt         2775 non-null   object
 2   death_yn              2775 non-null   object
dtypes: object(3)
memory usage: 86.7+ KB


In [6]:
# df['week_number'] = pd.to_datetime(df["cdc_case_earliest_dt"]).dt.to_period('W-Mon').dt.start_time.dt.strftime('%Y-%m-%d')
df["cdc_case_earliest_dt"] = pd.to_datetime(df["cdc_case_earliest_dt"])
df["year"] = df["cdc_case_earliest_dt"].dt.year
df["month"] = df["cdc_case_earliest_dt"].dt.month
df["year_month"] = df["cdc_case_earliest_dt"].astype(str).str[:7]
df['week_of_year'] = df['cdc_case_earliest_dt'].dt.isocalendar().week
df['week_range'] = df['cdc_case_earliest_dt'].dt.to_period('W-Mon').dt.start_time.dt.strftime('%Y-%m-%d') + ' to ' + df['cdc_case_earliest_dt'].dt.to_period('W-Mon').dt.end_time.dt.strftime('%Y-%m-%d')
df["week_number"] = df['cdc_case_earliest_dt'].dt.to_period('W-Mon').dt.start_time.dt.strftime('%Y-%m-%d')
df = df.replace(',','', regex=True)

In [7]:
df.reset_index(inplace = True, drop = True)

In [8]:
df.head()

,cdc_case_earliest_dt,cdc_report_dt,death_yn,year,month,year_month,week_of_year,week_range,week_number
0,2020-01-01,137,No,2020,1,2020-01,1,2019-12-31 to 2020-01-06,2019-12-31
1,2020-01-01,468,Yes,2020,1,2020-01,1,2019-12-31 to 2020-01-06,2019-12-31
2,2020-01-02,59,No,2020,1,2020-01,1,2019-12-31 to 2020-01-06,2019-12-31
3,2020-01-02,9,Yes,2020,1,2020-01,1,2019-12-31 to 2020-01-06,2019-12-31
4,2020-01-03,61,No,2020,1,2020-01,1,2019-12-31 to 2020-01-06,2019-12-31


Now it needs to make it aggregate it by week

In [9]:
df['cdc_report_dt'] = df['cdc_report_dt'].astype(str).astype(int)

In [10]:
df_groupby = df.groupby(['week_number', 'death_yn']).agg({
    'cdc_case_earliest_dt': 'first',
    'cdc_report_dt': 'sum',
    'year': 'first',
    'month': 'first',
    'year_month': 'first',
    'week_range': 'first',
    # Add more columns as needed
}).reset_index()

In [11]:
df_groupby.head()

,week_number,death_yn,cdc_case_earliest_dt,cdc_report_dt,year,month,year_month,week_range
0,2019-12-31,No,2020-01-01,579,2020,1,2020-01,2019-12-31 to 2020-01-06
1,2019-12-31,Yes,2020-01-01,505,2020,1,2020-01,2019-12-31 to 2020-01-06
2,2020-01-07,No,2020-01-07,227,2020,1,2020-01,2020-01-07 to 2020-01-13
3,2020-01-07,Yes,2020-01-07,18,2020,1,2020-01,2020-01-07 to 2020-01-13
4,2020-01-14,No,2020-01-14,197,2020,1,2020-01,2020-01-14 to 2020-01-20


In [13]:
df_groupby.to_csv('COVID-19_Case_Surveillance_Public_clean.csv')